In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.7.3.jar
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit,explode
import pyspark.sql.functions as f
spark=SparkSession.builder.appName("ETL job").config("spark.jars", "postgresql-42.7.3.jar").getOrCreate()

--2025-07-14 05:54:30--  https://jdbc.postgresql.org/download/postgresql-42.7.3.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1089312 (1.0M) [application/java-archive]
Saving to: ‘postgresql-42.7.3.jar.2’

postgresql-42.7.3.j 100%[===================>]   1.04M  5.98MB/s    in 0.2s    

2025-07-14 05:54:31 (5.98 MB/s) - ‘postgresql-42.7.3.jar.2’ saved [1089312/1089312]



In [ ]:
#Extract
df=spark.read.text("/content/wordData.txt")
df.show()

+--------------------+
|               value|
+--------------------+
|This is a Japanes...|
|The team members ...|
|As the years pass...|
|If you don't like...|
|He was disappoint...|
|When he encounter...|
|Situps are a terr...|
|Toddlers feeding ...|
|Edith could decid...|
|Her daily goal wa...|
|Tomorrow will bri...|
|His son quipped t...|
|He wondered why a...|
|If my calculator ...|
|The hummingbird's...|
|He went on a whis...|
|This is the last ...|
|I come from a tri...|
|The delicious aro...|
|Weather is not tr...|
+--------------------+
only showing top 20 rows



In [ ]:
#Transformation
df2=df.withColumn("splitedData", f.split("value"," "))
df3=df2.withColumn("words", explode("splitedData"))
wordDf=df3.select("words")
wordCount=wordDf.groupBy("words").count()

In [ ]:
#Load
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://database-1.cdmqwis866t7.ap-south-1.rds.amazonaws.com/"
table="spark_job_schema.word_counts"
user="postgres"
password="Nabeel_123"

In [ ]:
wordCount.write.format("jdbc").option("driver",driver).option("url",url).option("dbtable",table).option("mode","append").option("user",user).option("password",password).save()